In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('ceneval_data.csv', index_col=0, encoding='utf-8', low_memory=False)
df

,TIPO_EXA,ANO_NAC,SEXO,LENG_MA,LENG_PA,PLT_PROC,RAZ_RAPT,RAZ_RACT,RAZ_OPT,RAZ_PCUM,...,SER_TVP,SER_TABL,CUARTOS,SER_PC,SER_TV,SER_AUTO,SER_BANO,VAC_RM,DICTAMEN,VECES_EXAM
0,0.0077,0.90625,0,0.0,0.0,0.0355,0.0,1.0,1.0,0.0,...,1.0,1.0,0.22,1.0,1.0,1.0,1.0,0.0,0,1.0
1,0.0705,1.00000,1,0.0,0.0,0.0900,1.0,0.0,0.0,0.0,...,0.0,0.0,0.22,1.0,1.0,1.0,1.0,1.0,0,1.0
2,0.0705,0.90625,0,0.0,0.0,0.0900,1.0,1.0,1.0,0.0,...,0.0,0.0,0.41,0.0,1.0,0.0,1.0,0.0,1,1.0
3,0.0705,1.00000,1,0.0,0.0,0.0900,1.0,1.0,1.0,1.0,...,1.0,1.0,0.41,1.0,1.0,0.0,1.0,1.0,1,1.0
4,0.0705,0.84375,0,0.0,0.0,0.0900,1.0,1.0,1.0,0.0,...,0.0,0.0,0.22,1.0,1.0,0.0,1.0,0.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15608,0.0303,1.00000,1,0.0,0.0,0.0434,0.0,1.0,1.0,0.0,...,1.0,0.0,0.41,0.0,1.0,1.0,1.0,1.0,1,1.0
15609,0.0705,1.00000,1,0.0,0.0,0.0900,0.0,1.0,1.0,0.0,...,0.0,0.0,0.22,0.0,1.0,1.0,1.0,1.0,0,1.0
15610,0.0705,1.00000,1,0.0,0.0,0.0900,1.0,1.0,1.0,0.0,...,1.0,0.0,0.41,1.0,1.0,1.0,1.0,1.0,1,1.0
15611,0.0705,1.00000,1,0.0,0.0,0.0900,0.0,0.0,1.0,1.0,...,1.0,0.0,0.22,1.0,1.0,1.0,1.0,1.0,0,1.0


In [3]:
elements = df.size
elements

1857947

In [4]:
def generate_random_alpha():
    return (10 ** np.random.randint(-5,0)) * np.random.randint(1,10)

In [5]:
def generate_random_config(layer_sizes=100, batch='auto', n_samples=1500):
    # hidden_layer_sizes = (layer_sizes,)
    hidden_layer_sizes = tuple([np.random.randint(50, 200) for i in range(np.random.randint(1,10))])
    activation = [
        'identity',
        'logistic',
        'tanh',
        'relu'
    ]
    solver = [
        'lbfgs',
        'sgd',
        'adam'
    ]
    learning_rate = [
        'constant',
        'invscaling',
        'adaptive'
    ]
    alpha = np.random.rand()
    batch_size=batch if batch is 'auto' else (np.random.randint(10, 200), n_samples)
    learning_rate_init = generate_random_alpha()
    max_iter = np.random.randint(150, 300)
    momentum = generate_random_alpha()
    return { 
        'hidden_layer_sizes': hidden_layer_sizes,
        'activation': activation[np.random.randint(0,4)],
        'solver': solver[np.random.randint(0,3)],
        'learning_rate': learning_rate[np.random.randint(0,3)],
        'alpha': alpha,
        'batch_size': batch_size,
        'learning_rate_init': learning_rate_init,
        'max_iter': max_iter,
        'momentum': momentum
    }
    

In [6]:
def generate_models_config(samples, n_samples):
    return [generate_random_config(n_samples=n_samples) for i in range(samples)]

In [7]:
def find_better_model(x_train, y_train, x_test, y_test,models_config=[]):
    better_score = -1
    best_model = None
    for model_config in models_config:
        mlp = MLPClassifier(**model_config)
        mlp.fit(x_train, y_train)
        y_pred=mlp.predict(x_test)
        accuracy=accuracy_score(y_test, y_pred)
        if accuracy > better_score:
            better_score = accuracy
            best_model = mlp
    return mlp, better_score

In [8]:
def generate_random_layers():
    # return [tuple([np.random.randint(10, 300) for i in range(np.random.randint(1,100))]) for el in range(1000)]
    return [tuple([np.random.randint(10, 100) for i in range(np.random.randint(1,10))]) for el in range(1000)]

In [9]:
# Selecting features
y = df['DICTAMEN']
x = df.drop(["DICTAMEN"], axis=1).values 

# Spliting dataset into training set and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3) # 70% training and 30% test

In [10]:
x_train

array([[0.0583 , 0.84375, 1.     , ..., 1.     , 0.     , 1.     ],
       [0.0655 , 0.75   , 1.     , ..., 1.     , 0.     , 1.     ],
       [0.0346 , 0.96875, 1.     , ..., 1.     , 0.     , 1.     ],
       ...,
       [0.0411 , 0.78125, 0.     , ..., 1.     , 0.     , 1.     ],
       [0.0394 , 0.78125, 1.     , ..., 1.     , 0.     , 1.     ],
       [0.0411 , 0.875  , 0.     , ..., 1.     , 0.     , 1.     ]])

In [11]:
y_train

1403     1
7408     0
9466     1
3002     1
10333    0
        ..
8769     0
7430     1
11033    1
12922    1
10003    1
Name: DICTAMEN, Length: 10929, dtype: int64

In [514]:
mlp = MLPClassifier(**generate_random_config(n_samples=elements))

In [527]:
mlp.fit(x_train, y_train)

MLPClassifier(activation='logistic', alpha=0.9028682871300118,
              batch_size='auto', beta_1=0.9, beta_2=0.999, early_stopping=False,
              epsilon=1e-08, hidden_layer_sizes=(100,),
              learning_rate='constant', learning_rate_init=0.003, max_iter=168,
              momentum=0.6000000000000001, n_iter_no_change=10,
              nesterovs_momentum=True, power_t=0.5, random_state=None,
              shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
              verbose=False, warm_start=False)

In [528]:
y_pred=mlp.predict(x_test)

In [531]:
mlp.score(x_test, y_test)

0.591801878736123

In [532]:
# MLP Accuracy
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.591801878736123


In [547]:
models_config = generate_models_config(samples=1000, n_samples=elements)
models_config

[{'hidden_layer_sizes': (100,),
  'activation': 'identity',
  'solver': 'sgd',
  'learning_rate': 'invscaling',
  'alpha': 0.44587783092470923,
  'batch_size': 'auto',
  'learning_rate_init': 0.0007,
  'max_iter': 175,
  'momentum': 0.5},
 {'hidden_layer_sizes': (100,),
  'activation': 'identity',
  'solver': 'adam',
  'learning_rate': 'invscaling',
  'alpha': 0.08797414682989368,
  'batch_size': 'auto',
  'learning_rate_init': 0.004,
  'max_iter': 97,
  'momentum': 0.1},
 {'hidden_layer_sizes': (100,),
  'activation': 'tanh',
  'solver': 'adam',
  'learning_rate': 'adaptive',
  'alpha': 0.9455229627729469,
  'batch_size': 'auto',
  'learning_rate_init': 0.8,
  'max_iter': 57,
  'momentum': 0.00030000000000000003},
 {'hidden_layer_sizes': (100,),
  'activation': 'tanh',
  'solver': 'adam',
  'learning_rate': 'invscaling',
  'alpha': 0.3866672058040804,
  'batch_size': 'auto',
  'learning_rate_init': 0.02,
  'max_iter': 79,
  'momentum': 0.05},
 {'hidden_layer_sizes': (100,),
  'activat

In [548]:
best_model, better_score = find_better_model(x_train, y_train, x_test, y_test,models_config=models_config)
print("better Accuray", better_score)

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (97) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (16) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (86) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (187) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/co

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (65) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (17) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (84) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (121) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/co

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (30) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (124) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (118) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (36) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/c

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (122) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (13) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (96) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (116) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/c

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (36) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (105) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (80) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (29) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/co

...

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (18) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (58) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (33) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/con

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (27) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (33) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (37) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (180) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/co

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (17) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (183) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (113) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (61) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/c

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (91) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (124) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (11) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (101) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/c

better Accuray 0.6652433817250214


In [550]:
best_model.get_params()

{'activation': 'logistic',
 'alpha': 0.13611383704284163,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'adaptive',
 'learning_rate_init': 0.0007,
 'max_iter': 77,
 'momentum': 0.0004,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [574]:
models_layers = generate_random_layers()
models_layers

[(74, 48, 71, 87, 98, 75, 74, 77, 79),
 (28, 84, 53),
 (97, 83, 34),
 (97, 54),
 (44,),
 (25, 88, 75, 66, 37, 94, 69, 77, 36),
 (98, 19, 44, 39, 89, 63, 95, 31),
 (67, 60, 75, 93, 85),
 (38, 95, 34, 60, 34, 57, 27, 47),
 (25, 30, 14, 75),
 (96, 52, 70, 95, 74, 40),
 (36, 72),
 (36, 48, 88, 37, 33),
 (21, 45, 51),
 (80, 43, 83, 13, 50, 57),
 (15, 43, 91, 38, 19, 46, 95, 35),
 (18, 46, 92, 76, 48, 63),
 (79, 99),
 (24, 28, 46, 25, 47, 59, 98, 31),
 (72, 74, 25),
 (58, 22, 16, 54, 97, 39),
 (73, 43, 20, 63, 98, 73, 66, 24, 52),
 (14, 96, 15, 49, 28, 20),
 (33,),
 (78,),
 (93, 42, 68, 27, 47, 63, 78),
 (20, 50, 82, 50, 70, 68, 62, 43, 95),
 (14,),
 (64,),
 (15, 44, 51, 35, 10),
 (68, 29),
 (81, 14, 27, 26, 45, 84, 28),
 (32, 26, 65, 96, 15, 91, 79, 23, 79),
 (98, 49, 27, 95, 35, 72, 31),
 (28, 49, 79, 39),
 (98, 32, 84, 79),
 (29, 50, 22, 58, 68, 71, 91, 38, 22),
 (41, 95, 85),
 (72, 35, 87, 72, 31, 27, 54),
 (17, 69, 55),
 (40, 16, 63, 75, 67, 75),
 (56,),
 (47, 83, 91, 43, 13, 74),
 (57,

In [575]:
new_configurations = [best_model.get_params() for el in range(1000)]
new_configurations

[{'activation': 'logistic',
  'alpha': 0.13611383704284163,
  'batch_size': 'auto',
  'beta_1': 0.9,
  'beta_2': 0.999,
  'early_stopping': False,
  'epsilon': 1e-08,
  'hidden_layer_sizes': (100,),
  'learning_rate': 'adaptive',
  'learning_rate_init': 0.0007,
  'max_iter': 77,
  'momentum': 0.0004,
  'n_iter_no_change': 10,
  'nesterovs_momentum': True,
  'power_t': 0.5,
  'random_state': None,
  'shuffle': True,
  'solver': 'adam',
  'tol': 0.0001,
  'validation_fraction': 0.1,
  'verbose': False,
  'warm_start': False},
 {'activation': 'logistic',
  'alpha': 0.13611383704284163,
  'batch_size': 'auto',
  'beta_1': 0.9,
  'beta_2': 0.999,
  'early_stopping': False,
  'epsilon': 1e-08,
  'hidden_layer_sizes': (100,),
  'learning_rate': 'adaptive',
  'learning_rate_init': 0.0007,
  'max_iter': 77,
  'momentum': 0.0004,
  'n_iter_no_change': 10,
  'nesterovs_momentum': True,
  'power_t': 0.5,
  'random_state': None,
  'shuffle': True,
  'solver': 'adam',
  'tol': 0.0001,
  'validation_

In [576]:
for i in range(1000):
    new_configurations[i]['hidden_layer_sizes'] = models_layers[i]
    del(new_configurations[i]['max_iter'])
new_configurations

[{'activation': 'logistic',
  'alpha': 0.13611383704284163,
  'batch_size': 'auto',
  'beta_1': 0.9,
  'beta_2': 0.999,
  'early_stopping': False,
  'epsilon': 1e-08,
  'hidden_layer_sizes': (74, 48, 71, 87, 98, 75, 74, 77, 79),
  'learning_rate': 'adaptive',
  'learning_rate_init': 0.0007,
  'momentum': 0.0004,
  'n_iter_no_change': 10,
  'nesterovs_momentum': True,
  'power_t': 0.5,
  'random_state': None,
  'shuffle': True,
  'solver': 'adam',
  'tol': 0.0001,
  'validation_fraction': 0.1,
  'verbose': False,
  'warm_start': False},
 {'activation': 'logistic',
  'alpha': 0.13611383704284163,
  'batch_size': 'auto',
  'beta_1': 0.9,
  'beta_2': 0.999,
  'early_stopping': False,
  'epsilon': 1e-08,
  'hidden_layer_sizes': (28, 84, 53),
  'learning_rate': 'adaptive',
  'learning_rate_init': 0.0007,
  'momentum': 0.0004,
  'n_iter_no_change': 10,
  'nesterovs_momentum': True,
  'power_t': 0.5,
  'random_state': None,
  'shuffle': True,
  'solver': 'adam',
  'tol': 0.0001,
  'validation_

In [577]:
best_layers, best_score = find_better_model(x_train, y_train, x_test, y_test,models_config=new_configurations)
print("best Accuray", best_score)

best Accuray 0.6528608027327071


In [578]:
best_layers.get_params()

{'activation': 'logistic',
 'alpha': 0.13611383704284163,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (29, 56, 51, 14, 41, 40, 73, 11),
 'learning_rate': 'adaptive',
 'learning_rate_init': 0.0007,
 'max_iter': 200,
 'momentum': 0.0004,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

**attemp 2**

In [591]:
models_config_2 = generate_models_config(samples=1000, n_samples=elements)
models_config_2

[{'hidden_layer_sizes': (100,),
  'activation': 'logistic',
  'solver': 'adam',
  'learning_rate': 'invscaling',
  'alpha': 0.23808119625838287,
  'batch_size': 'auto',
  'learning_rate_init': 0.0002,
  'momentum': 0.08},
 {'hidden_layer_sizes': (100,),
  'activation': 'relu',
  'solver': 'sgd',
  'learning_rate': 'constant',
  'alpha': 0.5940146441477872,
  'batch_size': 'auto',
  'learning_rate_init': 0.06,
  'momentum': 2e-05},
 {'hidden_layer_sizes': (100,),
  'activation': 'tanh',
  'solver': 'lbfgs',
  'learning_rate': 'constant',
  'alpha': 0.5984067669659267,
  'batch_size': 'auto',
  'learning_rate_init': 0.30000000000000004,
  'momentum': 0.001},
 {'hidden_layer_sizes': (100,),
  'activation': 'logistic',
  'solver': 'adam',
  'learning_rate': 'invscaling',
  'alpha': 0.9302335701692407,
  'batch_size': 'auto',
  'learning_rate_init': 0.8,
  'momentum': 2e-05},
 {'hidden_layer_sizes': (100,),
  'activation': 'tanh',
  'solver': 'adam',
  'learning_rate': 'constant',
  'alpha'

In [592]:
best_model_2, better_score_2 = find_better_model(x_train, y_train, x_test, y_test,models_config=models_config_2)
print("better Accuray", better_score)

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/label.py:749: RuntimeWarning: invalid value encountered in greater
  y = np.array(y > threshold, dtype=np.int)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perce

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt

better Accuray 0.6652433817250214


In [598]:
print("better Accuray", better_score_2)

better Accuray 0.6675918018787361


In [593]:
best_model_2.get_params()

{'activation': 'logistic',
 'alpha': 0.41121801382173484,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.0009000000000000001,
 'max_iter': 200,
 'momentum': 0.001,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

**attemp 3**

In [14]:
models_config_3 = generate_models_config(samples=100, n_samples=elements)
models_config_3

[{'hidden_layer_sizes': (185, 63, 56, 58, 62, 155, 71, 55, 108),
  'activation': 'relu',
  'solver': 'adam',
  'learning_rate': 'constant',
  'alpha': 0.32378023344004303,
  'batch_size': 'auto',
  'learning_rate_init': 0.0004,
  'max_iter': 192,
  'momentum': 0.02},
 {'hidden_layer_sizes': (87,),
  'activation': 'identity',
  'solver': 'lbfgs',
  'learning_rate': 'adaptive',
  'alpha': 0.7491194221426849,
  'batch_size': 'auto',
  'learning_rate_init': 0.2,
  'max_iter': 294,
  'momentum': 0.05},
 {'hidden_layer_sizes': (60, 122, 109, 56, 147, 90, 57),
  'activation': 'tanh',
  'solver': 'lbfgs',
  'learning_rate': 'constant',
  'alpha': 0.7473658639471288,
  'batch_size': 'auto',
  'learning_rate_init': 0.002,
  'max_iter': 227,
  'momentum': 0.07},
 {'hidden_layer_sizes': (98, 185, 192, 73),
  'activation': 'tanh',
  'solver': 'sgd',
  'learning_rate': 'invscaling',
  'alpha': 0.5264780681217199,
  'batch_size': 'auto',
  'learning_rate_init': 0.05,
  'max_iter': 265,
  'momentum': 

In [ ]:
best_model_3, better_score_3 = find_better_model(x_train, y_train, x_test, y_test,models_config=models_config_3)
print("better Accuray", better_score_3)

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (249) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (217) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (258) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (235) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (172) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/label.py:749: RuntimeWarning: invalid value encountered in greater
  y = np.array(y > threshold, dtype=np.int)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (165) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (158) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptr

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (276) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (197) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (178) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (233) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (154) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (206) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (229) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (255) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (178) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (156) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (266) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (193) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (153) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (253) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (210) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (296) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt

In [ ]:
best_model_3.get_params()

**Using grid search**

In [13]:
def to_grid_search_data(configs: dict):
    return {
        'hidden_layer_sizes': [config['hidden_layer_sizes'] for config in configs],
        'activation': ['identity', 'logistic', 'tanh', 'relu'],
        'solver': ['lbfgs', 'sgd', 'adam'],
        'alpha': [config['alpha'] for config in configs],
        'learning_rate': ['constant', 'invscaling', 'adaptive']
    }

In [16]:
parameter_space = to_grid_search_data(models_config_3)
parameter_space

{'hidden_layer_sizes': [(185, 63, 56, 58, 62, 155, 71, 55, 108),
  (87,),
  (60, 122, 109, 56, 147, 90, 57),
  (98, 185, 192, 73),
  (93, 73, 80, 145, 116),
  (97, 119, 174),
  (193,),
  (127, 106),
  (107, 140, 150, 139, 64, 57),
  (155, 163, 138, 177, 178),
  (68, 85, 126, 64, 88, 86, 71, 72),
  (154, 128, 84, 142, 70, 153, 172, 151, 84),
  (132, 87, 51, 74, 140),
  (152, 152, 108, 55),
  (102,),
  (132, 114, 71, 56),
  (60, 71),
  (117, 97, 126, 197, 76, 141),
  (80, 173),
  (100, 99, 89, 74, 158, 138),
  (162, 63, 146, 106, 117),
  (180, 191, 50),
  (116, 71, 144, 194, 154, 51, 83, 94, 113),
  (83, 108, 111, 104, 132, 174, 87, 101, 75),
  (95, 145, 130, 137, 174, 58, 172, 90, 160),
  (137, 112, 170),
  (173, 92, 170, 126, 77, 115),
  (92, 183, 173, 60, 151),
  (88, 89, 150, 149, 103, 142),
  (163, 60, 97),
  (100, 84),
  (109, 69, 126),
  (134, 186),
  (84, 181, 101, 184, 71, 132),
  (149, 186, 156, 133, 147, 189),
  (189, 89, 189),
  (81, 95),
  (149, 64, 121, 59),
  (108, 197, 72

In [17]:
from sklearn.model_selection import GridSearchCV
mlp_no_config = MLPClassifier(max_iter=500)
clf = GridSearchCV(mlp_no_config, parameter_space, n_jobs=-1, cv=3)
clf.fit(x_train, y_train)

/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning

KeyboardInterrupt



In [ ]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
from sklearn.metrics import classification_report
y_true = y_test
y_pred = clf.predict(x_test)
print('Results on the test set:')
print(classification_report(y_true, y_pred))

In [ ]:
params_options = {
    'hidden_layer_sizes': [np.fill((i,), 100) for i in range(1,11)],
    'activation': ['logistic'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [np.random.rand() for i in range(0,100)],
    'learning_rate': ['constant', 'adaptive']
}

In [ ]:
mlp_no_config_2 = MLPClassifier(max_iter=1000)
clf_2 = GridSearchCV(mlp_no_config_2, params_options, n_jobs=-1, cv=3)
clf_2.fit(x_train, y_train)

In [ ]:
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf_2.cv_results_['mean_test_score']
stds = clf_2.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf_2.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [ ]:
y_true_2 = y_test
y_pred_2 = clf_2.predict(x_test)
print('Results on the test set:')
print(classification_report(y_true_2, y_pred_2))